In [0]:
from google.colab import files 
uploaded = files.upload()

Saving gender_submission.csv to gender_submission.csv
Saving test.csv to test.csv
Saving train.csv to train.csv


In [0]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn import preprocessing
import matplotlib.pyplot as plt
import seaborn as sns

In [0]:
#init training dataframe
data_train = pd.read_csv('train.csv')
data_test = pd.read_csv('test.csv')
data_test_y = pd.read_csv('gender_submission.csv')


In [0]:
#chech the state of the data
missing_training_count = data_train.isnull().sum()
print("# of rows in training data = " + str(len(data_train.index)))
print(missing_training_count)

missing_testing_count = data_test.isnull().sum()
print("# of rows in test data = " + str(len(data_test.index)))
print(missing_testing_count)

missing_y_count = data_test_y.isnull().sum()
print("# of rows in test data = " + str(len(data_test_y.index)))
print(missing_y_count)

# of rows in training data = 891
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64
# of rows in test data = 418
PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64
# of rows in test data = 418
PassengerId    0
Survived       0
dtype: int64


In [0]:
unique_Embarked=data_train['Embarked'].nunique()
unique_Tickets = data_train['Ticket'].nunique()
print(unique_Embarked)
print(unique_Tickets)


3
681


In [0]:
# drop columns: 
## passengerid becuase intuition tells me it is arbitrary
## name because intuition tells me it is arbitrary. Hypothetically there could be a discrimination factor, but that is outside of the scope
## cabin becuase there are too many null values
## ticket because there is too much variety 

data_train_dropped_columns = data_train.drop(['PassengerId', 'Name', 'Cabin', 'Ticket'],axis= 1)
missing_training_count = data_train_dropped_columns.isnull().sum()
print("# of rows in training data = " + str(len(data_train_dropped_columns.index)))
print(missing_training_count)

data_test_dropped_columns = data_test.drop(['PassengerId', 'Name', 'Cabin', 'Ticket'],axis= 1)
missing_testing_count = data_test_dropped_columns.isnull().sum()
print("# of rows in test data = " + str(len(data_test_dropped_columns.index)))
print(missing_testing_count)

data_test_y_dropped_column= data_test_y.drop('PassengerId',axis=1)
missing_y_count = data_test_y_dropped_column.isnull().sum()
print("# of rows in test data = " + str(len(data_test_y_dropped_column.index)))
print(missing_y_count)


# of rows in training data = 891
Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64
# of rows in test data = 418
Pclass       0
Sex          0
Age         86
SibSp        0
Parch        0
Fare         1
Embarked     0
dtype: int64
# of rows in test data = 418
Survived    0
dtype: int64


In [0]:
data_test_dropped_columns['Survived']= data_test_y_dropped_column
data_test_dropped_columns.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Survived
0,3,male,34.5,0,0,7.8292,Q,0
1,3,female,47.0,1,0,7.0000,S,1
2,2,male,62.0,0,0,9.6875,Q,0
3,3,male,27.0,0,0,8.6625,S,0
4,3,female,22.0,1,1,12.2875,S,1


In [0]:
#remove the null values
data_train_dropped_nulls = data_train_dropped_columns.dropna(how='any',axis=0)
missing_training_count = data_train_dropped_nulls.isnull().sum()
print("# of rows in training data = " + str(len(data_train_dropped_nulls.index)))
print(missing_training_count)

data_test_dropped_nulls = data_test_dropped_columns.dropna(how='any',axis=0)
missing_testing_count = data_test_dropped_nulls.isnull().sum()
print("# of rows in test data = " + str(len(data_test_dropped_nulls.index)))
print(missing_testing_count)
data_test_dropped_nulls.head()

# of rows in training data = 712
Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64
# of rows in test data = 331
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
Survived    0
dtype: int64


,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Survived
0,3,male,34.5,0,0,7.8292,Q,0
1,3,female,47.0,1,0,7.0000,S,1
2,2,male,62.0,0,0,9.6875,Q,0
3,3,male,27.0,0,0,8.6625,S,0
4,3,female,22.0,1,1,12.2875,S,1


In [0]:
#set int values for catagories in 'Embarked'
def set_embarked_cat(aDF, index):
  for i in range(0,len(aDF.index)):
    value = aDF.iloc[i,index]
    if  value == "S":
      aDF.iloc[i,index] =1 
    elif value =="C":
      aDF.iloc[i,index] =2 
    elif value =='Q': 
      aDF.iloc[i,index] =3
  return aDF

#set int values for catagories in 'Sex'
def set_sex_cat(aDF, index):
  for i in range(0,len(aDF.index)):
    value = aDF.iloc[i,index]
    if value =='male':
      aDF.iloc[i,index] = 1
    else:
      aDF.iloc[i,index] = 2
  return aDF

data_train_clean = set_embarked_cat(data_train_dropped_nulls,7)
data_train_clean = set_sex_cat(data_train_clean,2)

data_test_clean = set_embarked_cat(data_test_dropped_nulls,6)
data_test_clean = set_sex_cat(data_test_clean,1)

data_test_clean.head(10)




/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Survived
0,3,1,34.5,0,0,7.8292,3,0
1,3,2,47.0,1,0,7.0000,1,1
2,2,1,62.0,0,0,9.6875,3,0
3,3,1,27.0,0,0,8.6625,1,0
4,3,2,22.0,1,1,12.2875,1,1
5,3,1,14.0,0,0,9.2250,1,0
6,3,2,30.0,0,0,7.6292,3,1
7,2,1,26.0,1,1,29.0000,1,0
8,3,2,18.0,0,0,7.2292,2,1
9,3,1,21.0,2,0,24.1500,1,0


In [0]:
#set the data to be normalized
Y_train = data_train_clean['Survived']
X_train  = data_train_clean.drop('Survived',axis=1)
Y_test = data_test_clean['Survived']
X_test =  data_test_clean.drop('Survived',axis=1)

In [0]:
#actually normalize the data !
train_array = X_train.values
test_array = X_test.values

min_max_scaler = preprocessing.MinMaxScaler()

X_train_scaled = min_max_scaler.fit_transform(train_array)
X_test_scaled = min_max_scaler.fit_transform(test_array)

X_train_normalized = pd.DataFrame(X_train_scaled)
X_test_normalized = pd.DataFrame(X_test_scaled)

In [0]:
log_reg = LogisticRegression()
log_reg.fit(X_train_normalized,Y_train)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
predictions = log_reg.predict(X_test_normalized)


In [0]:
score = log_reg.score(X_test_normalized, Y_test)
print(score)


0.9425981873111783


In [0]:
cf = metrics.confusion_matrix(Y_test,predictions)
print(cf)

[[192  12]
 [  7 120]]
